In [ ]:
import pandas as pd

import datareader
from problemsolver import ProblemSolver
import solutions

In [ ]:
FILE_PREFERENCES = "testdata2.xlsx"
FILE_GROUPS_TO = "groepen_test.xlsx"

In [ ]:
groups_to = pd.read_excel(FILE_GROUPS_TO, index_col=0).to_dict(orient="index")
processor = datareader.VoorkeurenProcessor(FILE_PREFERENCES)
preferences = processor.process(all_to_groups=list(groups_to.keys()))
students_info = processor.get_students_meta_info()

In [ ]:
ps = ProblemSolver(
    preferences,
    students_info,
    groups_to,
)

prob = ps.run()

In [ ]:
sa = solutions.SolutionAnalyzer(prob, preferences, processor.input)

display(sa.display_groepsindeling())
display(sa.solution_performance)
display(sa.display_student_performance())
display(sa.display_satisfied_preferences())
sa.to_excel("groepsindeling.xlsx")

# Analysis

In [ ]:
solution_performance_overview = dict()
known_hashes = set()
relevant_solutions = []
optimization_options = [
    "studentsatisfaction",
    # "n_preferences",
    "weighted_preferences",
    "least_satisfied",
]
for optimize in optimization_options:
    for max_imbalance_boys_girls_year in range(1, 4):
        ps = ProblemSolver(
            preferences,
            students_info,
            groups_to,
            optimize=optimize,
            max_imbalance_boys_girls_year=max_imbalance_boys_girls_year,
        )
        prob = ps.run()
        sa = solutions.SolutionAnalyzer(prob, preferences, processor.input)
        new_hash = sa.get_hash()
        if new_hash not in known_hashes:
            known_hashes.add(new_hash)
            relevant_solutions.append(sa)
            print(f"{optimize=} {max_imbalance_boys_girls_year=}")
            display(sa.display_groepsindeling())

            solution_performance_overview[(optimize, max_imbalance_boys_girls_year)] = (
                sa.solution_performance
            )

In [ ]:
pd.DataFrame.from_dict(solution_performance_overview, orient="index")

In [ ]:
for sol in relevant_solutions:
    display(sol.display_groepsindeling())
    display(sol.solution_performance)
    display(sol.display_student_performance())
    print("-" * 30)